In [1]:
# Neuron-synapse-astrocyte-interaction Simulation

#-----------------------------------------------------------------------------------------------------------------------
# Default Imports
#-----------------------------------------------------------------------------------------------------------------------
import numpy as np
import scipy.constants as spc

#-----------------------------------------------------------------------------------------------------------------------
# Brian2 import: we use Brian CPP-standalone code generation for fast parallelized simulations
#-----------------------------------------------------------------------------------------------------------------------
from brian2 import *
code_dir = './codegen'
prefs.GSL.directory = '/opt/anaconda3/envs/Brian2_NGILab/include/'   ## The directory where the GSL library headings are found
set_device('cpp_standalone',directory=code_dir,build_on_run=False)
prefs.devices.cpp_standalone.openmp_threads = 2 ## The number of threads used in the parallelization (machine-dependent)
prefs.logging.file_log = False
prefs.logging.delete_log_on_exit = True

import matplotlib.pyplot as plt

In [2]:
#-----------------------------------------------------------------------------------------------------------------------
## Utilities
#-----------------------------------------------------------------------------------------------------------------------
def varargin(pars, **kwargs):
    """
    varargin-like option for user-defined parameters in any function/module
    Use:
    pars = varargin(pars,**kwargs)

    Input:
    - pars     : the dictionary of parameters of the calling function
    - **kwargs : a dictionary of user-defined parameters

    Output:
    - pars     : modified dictionary of parameters to be used inside the calling
                 (parent) function

    Maurizio De Pitta', The University of Chicago, August 27th, 2014.
    """
    for key, val in kwargs.items():
        if key in pars:
            pars[key] = val
    return pars

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------
## Build User-defined convenience functions to be also called by equations in neuron models by Brian  
#-----------------------------------------------------------------------------------------------------------------------
def Hill(x,K,n):
    return x**n/(x**n+K**n)

Hill = Function(Hill,arg_units=[mmolar,mmolar,1], return_unit=1,auto_vectorise=False)
Hill_cpp = '''
    #include <math.h>
    double Hill(double x,double K,double n)
    {
        return pow(x,n)/(pow(x,n)+pow(K,n));
    };
    '''
Hill.implementations.add_implementation('cpp',Hill_cpp,compiler_kwds={'headers': ['"math.h"']})


def ThermalVoltage(T):
    return spc.R*(T+273.15)/spc.physical_constants['Faraday constant'][0]

ThermalPotential = Function(ThermalVoltage,arg_units=[1], return_unit=1,auto_vectorise=False)
ThermalVoltage_cpp = '''
    #include <gsl/gsl_const_mksa.h>
    double ThermalPotential(const double T)
    {
        const double R = GSL_CONST_MKSA_MOLAR_GAS;
        const double F = GSL_CONST_MKSA_FARADAY;
        return R*(T+273.15)/F;
    }
    '''
ThermalPotential.implementations.add_implementation('cpp',ThermalVoltage_cpp,
                                                  compiler_kwds={'headers': ['"gsl_const_mksa.h"'],
                                                                 'include_dirs': ['/opt/anaconda3/envs/Brian2_NGILab/include/gsl/']})


def NernstPotential(x_e,x_i,z,T):
    """
    Nernst potential in volts (w/out units)

    Input parameters (w/out units):
    - x_e : float   Intracellular concentration
    - x_i : float   Extracellyular concentration
    - z   : int     Ion valence
    - T   : float   Temperature in ^C

    Return:
    - E_x : Nernst Reverse Potential in volt (W/OUT units)
    """
    V_T = ThermalVoltage(T)
    return V_T/z*np.log(x_e/x_i)

NernstPotential = Function(NernstPotential,arg_units=[mmolar,mmolar,1,1], return_unit=1,auto_vectorise=False)
NernstPotential_cpp = '''
    #include <gsl/gsl_const_mksa.h>
    const double R = GSL_CONST_MKSA_MOLAR_GAS; 
    const double F = GSL_CONST_MKSA_FARADAY;
    double ThermalVoltage(const double T)
    {
        return R*(T+273.15)/F;
    };
    double NernstPotential(double x_e,double x_i,double z,const double T)
    {
        return ThermalVoltage(T)*log(x_e/x_i)/z;
    };
    '''
NernstPotential.implementations.add_implementation('cpp',NernstPotential_cpp,
                                                   dependencies={'log': DEFAULT_FUNCTIONS['log']},
                                                   compiler_kwds={'headers': ['"gsl_const_mksa.h"'],
                                                                  'include_dirs': ['/usr/include/gsl']})

In [ ]:
def neurons_parameters(model = '', **kwargs):
    # At first safe a dictionary with the single parameters needed for our neurons
    params ={
        # here we have to fit in the available parameters (example for structure is given)
        'key': 2233*unit,
        'key1': 233*unit
    } 

    params = varargin(params, **kwargs)

    # We define this quantities after any user-defined parameters are specified
    pars['T_adj'] = 2.3**(0.1*(pars['T_exp']-21))
    pars['E_Na'] = NernstPotential(pars['N0_e'],pars['N0_i'],1,pars['T_exp'])*volt
    pars['E_K']  = NernstPotential(pars['K0_e'],pars['K0_i'],1,pars['T_exp'])*volt
    pars['E_Cl'] = NernstPotential(pars['C0_e'],pars['C0_i'],-1,pars['T_exp'])*volt

    ## Physics constants
    pars['F'] = spc.physical_constants['Faraday constant'][0]*coulomb/mole
    
    return params

def synapse_parameters():
    x
    return params

def astrocyte_parameters():
    return params



In [ ]:
neurons_eqs = {'''
dv/dt = (g_l*(E_l-v) + g_e*(E_e-v) + g_i*(E_i-v) + I_ex)/C_m : volt (unless refractory)
dg_e/dt = -g_e/tau_e : siemens # post-synaptic exc. conductance
dg_i/dt = -g_i/tau_i : siemens # post-synaptic inh. conductance'''
}

synapse_eqs = {'''
du_S/dt = -Omega_f * u_S : 1 (event-driven) # Usage of releasable neurotransmitter per single action potential: 
dx_S/dt = Omega_d *(1 - x_S) : 1 (event-driven) # Fraction of synaptic neurotransmitter resources available: 
'''}

In [ ]:
def Neurons(N_e, N_i, params, eqs, model='hh-neuron',name='hh*',dt=None):
    neurons = NeuronGroup(N_e + N_i, eqs, threshold='v>v_thr', namespace=params, name=name, 
                          dt=dt, refractory= 'tau_r', method = 'euler', reset = 'v = V_r')
    
    
    neurons.v = E_l + rand()*(V_th-E_l)
    
    exc_neurons = neurons[:N_e]
    inh_neurons = neurons[N_e:]
    

    return exc_neurons, inh_neurons

In [ ]:
def Simulation(N_e, N_i):
    Neurons(N_e, N_i, pa)